# 整理

規則:
 - 上市
 - ETF、ETN、存託憑證、不動產投資信託證券不要
 - 四年內不得有注意或處置紀錄
 - 四年內有暫停紀錄
 - 自 2018 年開市後，取 5 年做訓練資料，2023 開市後，作為測試資料

In [1]:
import pandas as pd
import datetime

## 注意股

In [2]:
notices = pd.read_csv('TWSE_注意股_1080801至1120718.csv', sep=',', quotechar='"', skipinitialspace=True, escapechar='=', skiprows=1, encoding='utf-8')
notices = notices[['證券代號', '證券名稱']].dropna().drop_duplicates().reset_index(drop=True)
notices

,證券代號,證券名稱
0,00642U,元大S&P石油
1,00672L,元大S&P原油正2
2,00673R,元大S&P原油反1
3,00677U,富邦VIX
4,00715L,街口布蘭特油正2
...,...,...
1347,9944,新麗
1348,9945,潤泰新
1349,9946,三發地產
1350,9955,佳龍


## 處置股

In [3]:
punishes = pd.read_csv('TWSE_處置股_1080801至1120718.csv', sep=',', quotechar='"', skipinitialspace=True, escapechar='=', skiprows=1, encoding='utf-8')
punishes = punishes[['證券代號', '證券名稱']].dropna().drop_duplicates().reset_index(drop=True)
punishes['證券代號'] = punishes['證券代號'].astype(str).str.replace('"', '')
punishes

,證券代號,證券名稱
0,00642U,元大S&P石油
1,00672L,元大S&P原油正2
2,00677U,富邦VIX
3,00715L,街口布蘭特油正2
4,030001,萬海中信0B購01
...,...,...
441,9928,中視
442,9935,慶豐富
443,9944,新麗
444,9946,三發地產


## 暫停交易

In [4]:
sus = pd.read_csv('TWSE_暫停交易_1080801至1120718.csv', sep=',', quotechar='"', skipinitialspace=True, escapechar='=', skiprows=1, encoding='utf-8')
sus = sus[['證券代號', '證券名稱']].dropna().drop_duplicates().reset_index(drop=True)
sus['證券代號'] = sus['證券代號'].astype(str).str.replace('"', '')
sus

,證券代號,證券名稱
0,03005P,欣興元大15售14
1,03006P,欣興元大17售03
2,030072,國巨群益0A購01
3,03009P,國巨永豐91售03
4,03013P,欣興國票16售02
...,...,...
2110,8497,格威傳媒
2111,9110,越南控-DR
2112,911608,明輝-DR
2113,912398,友佳-DR


## 期間前就申請上市的公司 (如果在要取用期間上市之個股，代表沒有完整期間，不取用)

In [5]:
apply = pd.read_csv('TWSE_申請上市.csv', sep=',', quotechar='"', skipinitialspace=True, escapechar='=', skiprows=1, encoding='utf-8')

def convert_code(code):
    try:
        return str(int(code))
    except:
        return code

def convert_republican_date(date):
    return datetime.datetime.strptime(f'{str(int(date.split("/")[0]) + 1911)}/{str(date.split("/")[1])}/{str(date.split("/")[2])}', '%Y/%m/%d').strftime('%Y-%m-%d')

apply['公司代號'] = apply['公司代號'].apply(lambda x: convert_code(x))

apply = apply[['公司代號', '股票上市買賣日期']].dropna().copy()
apply['股票上市買賣日期'] = apply['股票上市買賣日期'].apply(lambda x: convert_republican_date(x))
apply

,公司代號,股票上市買賣日期
9,2432,2023-05-31
10,6902,2023-07-13
14,6657,2023-06-12
16,6869,2023-03-14
17,6873,2023-03-06
...,...,...
738,1533,2001-04-06
739,2445,2001-04-02
740,9939,2001-03-02
741,1729,2001-03-21


## 處理股票資料

### 取得預備的上市股目錄

In [6]:
f = pd.ExcelFile('台股上市目錄_1120718.xlsx')
f

In [7]:
f.sheet_names

['水泥工業',
 '食品工業',
 '塑膠工業',
 '紡織纖維',
 '電機機械',
 '電器電纜',
 '生技醫療業',
 '化學工業',
 '玻璃陶瓷',
 '造紙工業',
 '鋼鐵工業',
 '橡膠工業',
 '汽車工業',
 '電腦及周邊設備業',
 '半導體業',
 '電子零組件業',
 '其他電子業',
 '通信網路業',
 '資訊服務業',
 '建材營造業',
 '航運業',
 '觀光餐旅',
 '銀行業',
 '保險業',
 '金控業',
 '貿易百貨業',
 '光電業',
 '電子通路業',
 '數位雲端',
 '證券業',
 '綠能環保',
 '其他業',
 '運動休閒',
 '油電燃氣業',
 '居家生活',
 'ETF',
 '不動產投資信託證券',
 '資產基礎證券',
 'ETN',
 '存託憑證']

### 篩選要取用的股票類別

In [8]:
cat = []
for item in f.sheet_names:
    # ETF 及 ETN 不要
    if item == 'ETF' or item == 'ETN':
        continue
    # 不動產投資信託證券、存託憑證不要
    if item == '不動產投資信託證券' or item == '存託憑證':
        continue
    cat.append(item)
cat

['水泥工業',
 '食品工業',
 '塑膠工業',
 '紡織纖維',
 '電機機械',
 '電器電纜',
 '生技醫療業',
 '化學工業',
 '玻璃陶瓷',
 '造紙工業',
 '鋼鐵工業',
 '橡膠工業',
 '汽車工業',
 '電腦及周邊設備業',
 '半導體業',
 '電子零組件業',
 '其他電子業',
 '通信網路業',
 '資訊服務業',
 '建材營造業',
 '航運業',
 '觀光餐旅',
 '銀行業',
 '保險業',
 '金控業',
 '貿易百貨業',
 '光電業',
 '電子通路業',
 '數位雲端',
 '證券業',
 '綠能環保',
 '其他業',
 '運動休閒',
 '油電燃氣業',
 '居家生活',
 '資產基礎證券']

### 將資料轉變為股票代號、名稱對應類別的 DataFrame

In [9]:
data = pd.read_excel('台股上市目錄_1120718.xlsx', sheet_name=cat)

In [10]:
ind = pd.DataFrame()
for d in data.items():
    d[1]['類別'] = d[0]
    ind = pd.concat([ind, d[1]], axis=0)

In [11]:
ind

,代號,名稱,類別
0,1101,台泥,水泥工業
1,1102,亞泥,水泥工業
2,1103,嘉泥,水泥工業
3,1104,環泥,水泥工業
4,1108,幸福,水泥工業
...,...,...,...
6,8482,商億-KY,居家生活
7,9911,櫻花,居家生活
8,9924,福興,居家生活
9,9934,成霖,居家生活


### 抓取期間內上市的股票

In [12]:
apply[apply['股票上市買賣日期'] > '2018/02/21']['公司代號']

9      2432
10     6902
14     6657
16     6869
17     6873
       ... 
107    6672
108    4564
109    8104
111    4967
113    6669
Name: 公司代號, Length: 82, dtype: object

### 列出個股之篩選條件

In [13]:
ind['注意'] = ind['代號'].isin(notices['證券代號'])
ind['處置'] = ind['代號'].isin(punishes['證券代號'])
ind['暫停'] = ind['代號'].isin(sus['證券代號'])
ind['availability'] = ind['代號'].isin(apply[apply['股票上市買賣日期'] > '2018/02/21']['公司代號'].astype(int))
ind['availability'] = ~ind['availability']
ind

,代號,名稱,類別,注意,處置,暫停,availability
0,1101,台泥,水泥工業,False,False,False,True
1,1102,亞泥,水泥工業,False,False,False,True
2,1103,嘉泥,水泥工業,False,False,False,True
3,1104,環泥,水泥工業,False,False,False,True
4,1108,幸福,水泥工業,False,False,False,True
...,...,...,...,...,...,...,...
6,8482,商億-KY,居家生活,False,False,False,True
7,9911,櫻花,居家生活,False,False,False,True
8,9924,福興,居家生活,False,False,False,True
9,9934,成霖,居家生活,False,False,False,True


In [14]:
ind['注意'].value_counts()

注意
False    978
True      26
Name: count, dtype: int64

In [15]:
ind['處置'].value_counts()

處置
False    1001
True        3
Name: count, dtype: int64

In [16]:
ind['暫停'].value_counts()

暫停
False    1003
True        1
Name: count, dtype: int64

In [17]:
ind['availability'].value_counts()

availability
True     922
False     82
Name: count, dtype: int64

 > 注意股其實也包含處置股

### 丟掉有注意或處置紀錄或暫停交易紀錄或期間內上市的股票

In [18]:
ind = ind[~(ind['注意'] | ind['處置'] | ind['暫停'] | ~ind['availability'])].copy()
ind[['代號', '名稱', '類別']].apply(str)
ind

,代號,名稱,類別,注意,處置,暫停,availability
0,1101,台泥,水泥工業,False,False,False,True
1,1102,亞泥,水泥工業,False,False,False,True
2,1103,嘉泥,水泥工業,False,False,False,True
3,1104,環泥,水泥工業,False,False,False,True
4,1108,幸福,水泥工業,False,False,False,True
...,...,...,...,...,...,...,...
6,8482,商億-KY,居家生活,False,False,False,True
7,9911,櫻花,居家生活,False,False,False,True
8,9924,福興,居家生活,False,False,False,True
9,9934,成霖,居家生活,False,False,False,True


### 輸出結果

In [19]:
ind[['代號', '名稱', '類別']].to_csv('ind.csv', index=False)